In [1]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime

In [2]:
def pyramid(image, scale=1.5, minSize=(30, 30)):
    yield image
    while True:
        w = int(image.shape[1] / scale)
        image = imutils.resize(image, width=w)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break
        yield image
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
(winW, winH) = (120,24)

clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def show(img, x, y, winW, winH):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def global_predict():
    global glob_X, glob_Y
    pred=clf.predict(glob_X)
    if glob_Y.size == 0:
        glob_Y=pred
    else:
        glob_Y=np.concatenate((glob_Y,pred))
    glob_X=[]
    
def predict(filename, scene_name, label_resolution, rects):
    global glob_X,glob_IDX,glob_IT,glob_DUMP_IT
    
    image=imread_resized(scene_name, filename, label_resolution)
    channels=read_channels(scene_name, filename)
    X=[]
    for resized in pyramid(image, scale=1.5):
        for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=channels[y:y + winH, x:x + winW]
            features=channels_window.flatten()
            glob_X.append(features)
            glob_IDX.append([scene_name, filename, resized.shape, (x,y)])
            #show(resized, x, y, winW, winH)
    glob_IT+=1
    if glob_IT==glob_DUMP_IT:
        global_predict()
        glob_IT=0

In [5]:
glob_X=[]
glob_IDX=[]
glob_Y=np.array([])
glob_IT=0
glob_DUMP_IT=50 # adjust manually to RAM

[predict(*row) for row in tqdm(walk_dataset())]
global_predict()

now = datetime.now()
date_time = now.strftime("%d-%m-%Y_%H:%M:%S")
np.save("sliding_window_IDX_"+date_time,glob_IDX)
np.save("sliding_window_Y_"+date_time,glob_Y)
print(len(glob_IDX), len(glob_Y),len(glob_X))

#glob_IDX=np.load("sliding_window_IDX_07-04-2020_17:59:14.npy",allow_pickle=True)
#glob_Y=np.load("sliding_window_Y_07-04-2020_17:59:14.npy",allow_pickle=True)
glob_Y=np.array([[e] for e in glob_Y])
RES=np.append(glob_IDX, glob_Y, 1)
print(np.array(glob_IDX).shape, glob_Y.shape, RES.shape)

100%|██████████| 2092/2092 [04:35<00:00,  7.59it/s]


3029922 3029922 0


In [10]:
def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    img=imread_resized(scene_name, filename, label_resolution)
    file_res=RES[np.where((RES[:,0] == scene_name) * (RES[:,1] == filename))]
    print(file_res)

draw_predicted_rectangles(*list(walk_dataset())[0])

[['blue_1' 'IMG_20200229_075347.jpg' (780, 1040, 3) (0, 0) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' (780, 1040, 3) (32, 0) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' (780, 1040, 3) (64, 0) 0.0]
 ...
 ['blue_1' 'IMG_20200229_075347.jpg' (101, 136, 3) (0, 0) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' (101, 136, 3) (0, 32) 0.0]
 ['blue_1' 'IMG_20200229_075347.jpg' (101, 136, 3) (0, 64) 0.0]]


In [ ]:
def predict_single():
    row=walk_dataset()[0]
    predict(*row)
    
timeit.timeit(predict_single, number=10)
# window only - 1.36s
# features only - 1.47s
# classify per image - 2.82s
# classify per scale - 8.47s
# dsiplay, features only - 52.64s